# LoRaWAN Class-A End-Device

In this tutorial, we want to book one Pycom board and programm it as LoRaWAN Class-A Device.

## 1. Provision End-Device on LoRaWAN Server

Log in with your LoRaWAN account and password on the [LoRaWAN server](https://inetlab-lorawan.icube.unistra.fr) (ChirpStack) 

1. Create a **Device-Profile** for a Class-A  End-Device with OTAA mode (see [Documention](https://inetlab.icube.unistra.fr/index.php/Documentation/Doc-LRP_IoT-organisation#Comment_cr.C3.A9er_un_profil_d.27acc.C3.A8s_r.C3.A9seau_.3F_.28Device-profiles.29))

* Device-profile name : class-A-otaa-1.0.2 
* Network-server : chirpstack-network-server 
* LoRaWAN MAC version : 1.0.2
* LoRaWAN Regional Parameters revision : A 
* ADR algorithm : Default
* Max EIRP : 14
* Uplink interval (seconds) : 86400 
* Device supports OTAA : True 
* CLASS B : False
* CLASS C : False
* CODEC : Cayenne LPP

2. Create an **Application** called **class_a_app** (see [Documentation](https://inetlab.icube.unistra.fr/index.php/Documentation/Doc-LRP_IoT-organisation#Comment_cr.C3.A9er_une_application_.3F_.28Applications.29s)) 

3. Create an **End-Device** called **my_class_a_obj** inside **class_a_app** application (See [Documentation](https://inetlab.icube.unistra.fr/index.php/Documentation/Doc-LRP_IoT- organisation#Comment_ajouter_des_End-Devices_.C3.A0_une_application_.3F))

* **Note:** Generate a **random Device EUI** 

## 2. Configure the End-Device firmware


1. Copy template file for eb_demo example :

In [ ]:
!cp /home/jovyan/work/training/micropython-tutorials/lorawan/lopy-example/eb_demo/config.py.example /home/jovyan/work/training/micropython-tutorials/lorawan/lopy-example/eb_demo/config.py

2. Edit /home/jovyan/work/training/micropython-tutorials/lorawan/lopy-example/eb_demo/config.py 

Change **APP_KEY** and  **DEV_EUI** values according your End-Device **my_class_a_obj** declaration provisionned on the LoRaWAN Server.

```
""" Application configuration parameters """

# Time to wait in sec between msg
SLEEP_TIMER = 30

# LoRaWAN Settings
APP_EUI = 'AB CD AB CD 12 34 12 34'
APP_KEY = 'ff ff ff ff ff ff ff ff ff ff ff ff ff ff ff ff'
DEV_EUI = 'ff ff ff ff ff ff ff ff'
ADR = False
DATARATE = 5

# Leds colors
COLOR_YELLOW = 0xffff00
COLOR_RED = 0xff0000
COLOR_GREEN = 0x00ff00
COLOR_BLUE = 0x0000ff
COLOR_BROWN = 0x996633
COLOR_BLACK = 0x000000
```

## 3. Node Reservation

1. Book one Pycom node on the IoT-LAB Strasbourg site. 

**Note:** You can choose :

* ARCHI=pycom:fipy:eb31  (20 nodes)

or 

* ARCHI=pycom:lopy4:eb31 (10 nodes)

In [ ]:
%env ARCHI=pycom:fipy:eb31
%env SITE=strasbourg

2. Submit an experiment using the following command, 

**Note:** run only one experiment at a time, check experiment status on [IoT-LAB dashboard](https://www.iot-lab.info/testbed/dashboard)

In [ ]:
!iotlab-experiment submit -n "class-a" -d 60 -l 1,archi=$ARCHI+site=$SITE,

3. Wait for the experiment to be in the Running state:

In [ ]:
!iotlab-experiment wait --timeout 30 --cancel-on-timeout

4. Get the experiment nodes list:

In [ ]:
!iotlab-experiment --jmespath="items[*].network_address | sort(@)" get --nodes

**Be careful**, you must note the id of the booked node because it will be useful when you read its serial port. Each node is listed using its network address, which is of the form `pycom-<id>.<site>.iot-lab.info`. You have to remember the `pycom-<id>` string for the next command.

## 4. Flash the board

1. Open a first JupyterLab Terminal (File > New > Terminal) and copy/paste the following command. 

**NOTE:** Replace `pycom-<id>` and `<site>` with the right value. Keep this terminal open.

```
ssh -L 20000:pycom-<id>:20000 $IOTLAB_LOGIN@<site>.iot-lab.info
```

2. Open a second JupyterLab Terminal (File > New > Terminal) and copy/paste the following command. This command forwards remote serial port on JupyterLab in order to be able to flash firmware on the node.

```
socat -d -d pty,link=/tmp/ttyS0,echo=0,crnl TCP:127.0.0.1:20000s
```

3. Synchronize files with Rshell. It is quiet long (could be serveral minutes). **Please wait until the cell is completedly done.**

In [ ]:
!/home/jovyan/.local/bin/rshell -n -p /tmp/ttyS0 rsync /home/jovyan/work/training/micropython/lorawan/lopy-example/eb_demo/ /flash

## 4. Monitor Join Request

1. Close socat on second terminal (CTRL + C) in order to release serial port. Remember serial port access is exclusive, we can not open two connexion at the same time.

2. Go to [IoT-LAB dashboard](https://www.iot-lab.info/testbed/dashboard)

3. Go inside your experiment runnning 

4. Reset the your Pycom board (reset button)

5. Open Serial Port from dashboard (Terminal button), observe MicroPython messages logs.

6. Go to [LoRaWAN server](https://inetlab-lorawan.icube.unistra.fr), navigate to Applications > **class_a_app** >  **my_class_a_obj** > Device Data and LoRaWAN Frames and find the exact messages corresponding the OTAA phase (Join Request/ Join-Accept). Find the DevEUI and DevAddr value for your Device.

## 5. Monitor Uplink Messages 

1. Observe the Uplink Messages, what is the time interval between two Uplinks ? Could you explain with the MicroPython code ?

2. How many frequency are used for Uplink Messages ? (wait for enough history messages)

3. How many LoRaWAN Gateways see the same Uplink Messages ?

4. Can you observe other messages (Downlink or Status). What kind of messages is it ?


## 6. Send Downlink Messages

Envoyer un Downlink sans acquittement via l’interface ChirpStack avec le chaine en base64 suivante : dGl0aXRvdG8= . Observer les échanges de messages dans l’onglet LoRaWAN Frames. Vérifier la réception du message Downlink sur l’objet.

1. Send an **unconfirmed downlink** with [LoRaWAN server](https://inetlab-lorawan.icube.unistra.fr) , navigate to Applications > **class_a_app** >  **my_class_a_obj** >  Details > Enqueue downlink payload . Use paramaters: 

* port: **1**
* Confirmed downlink: **False**
* base64 encoded: **dGl0aXRvdG8=**

2. Verify reception on End-Device serial port via IoT-LAB Dashboard (terminal). What is message send to End-Device ?

3. Try to send a **confirmed downlink** :

* port: **1**
* Confirmed downlink: **True**
* base64 encoded: **dGl0aXRvdG8=**

4. Verify reception on End-Device serial port via IoT-LAB Dashboard (terminal). What is message send to End-Device ? Do have a confirmation of the End-Device Acknowledgment ?


## 7. Free up the resources

1. Since you finished the training, stop your experiment to free up the experiment nodes:

In [ ]:
!iotlab-experiment stop

## 8. References

* https://www.iot-lab.info/docs/getting-started/introduction/ 
* https://inetlab.icube.unistra.fr/index.php/Documentation/Doc-LRP_IoT-organisation 
* https://www.chirpstack.io/application-server/ 
* https://www.iot-lab.info/docs/boards/pycom/
* https://docs.pycom.io 
* https://micropython.fr/05.outils/terminal_serie/rshell/ 
* https://github.com/dhylands/rshell